<a href="https://colab.research.google.com/github/sahana-github/Machine-learning-Demo/blob/main/ML_Pipelines_with_ZenML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lesson 1.1: ML Pipelines with ZenML


In [ ]:
%pip install "zenml [server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2 #required for colab
%pip install ipython

# Restart the kernel using the following command:
import os
os._exit(0)

⠹ Installing integrations...


In [1]:
NGROK_TOKEN = "2b7iDypIKYd6hTnPDs7jvb9fTgs_4BrQZEpyAQ432XkfqBL2e"

In [2]:
from zenml.environment import Environment

if Environment.in_google_colab(): #colab only setup
#install and authenticate ngrok
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ZenML Setup

In [3]:
!rm -rf .zen
!zenml init

Initializing the ZenML global configuration version to 0.67.0
⠋ Initializing ZenML repository at /content.
Creating database tables
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
Creating default workspace 'default' ...
Creating default stack in workspace default...
⠧ Initializing ZenML repository at /content.
⠏ Initializing 

In [5]:
import numpy as np
from sklearn.base import ClassifierMixin # Changed import statement to correct module
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
  digits= load_digits()
  data =digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(
      data, digits.target, test_size = 0.2, shuffle=False
  )
  model = SVC(gamma=0.001)
  model.fit(X_train, y_train)
  test_acc= model.score(X_test, y_test)
  print(f"Test accuracy: (test_acc)")

train_test()

Test accuracy: (test_acc)


In [6]:
from string import digits
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple
@step
def importer() -> Tuple[
  Annotated[np.ndarray, "X_train"],
  Annotated [np.ndarray, "X_test"],
  Annotated [np.ndarray, "y_train"],
  Annotated [np.ndarray, "y_test"],
]:
#Load the digits dataset as numpy arrays.
  digits=load_digits()
  data= digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test= train_test_split(
      data, digits.target, test_size= 0.2, shuffle=False
  )
  return X_train, X_test, y_train, y_test

@step
def svc_trainer(
    X_train: np.ndarray,  # Corrected variable name
    y_train: np.ndarray,  # Corrected variable name
) -> ClassifierMixin:  # Corrected return type
    """Trains an SVC model."""
    model = SVC(gamma=0.001)  # Corrected typo in gamma
    model.fit(X_train, y_train)
    return model


@step
def evaluator(
    X_test: np.ndarray,  # Corrected variable name
    y_test: np.ndarray,  # Corrected variable name
    model: ClassifierMixin,
) -> float:
    """Evaluates the model and prints the test accuracy."""
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")  # Corrected print statement
    return test_acc

In [9]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  """Links all the steps together in a pipeline."""
  X_train, X_test, y_train, y_test = importer()
  # Pass X_train directly to the trainer, removing the subtraction
  model = svc_trainer(X_train=X_train, y_train=y_train)
  evaluator(X_test=X_test, y_test=y_test, model=model)

Running ZenML Pipelines

In [10]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Registered new pipeline: digits_pipeline.
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Step importer has started.
Failed to run step importer after 1 retries. Exiting.
imagen
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_bunch.py", line 54, in __getattr__
    return self[key]
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_bunch.py", line 39, in __getitem__
    return super().__getitem__(key)
KeyError: 'imagen'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/zenml/orchestrators/step_launcher.py", line 292, in launch
    self._run_step(
  File "/usr/local/lib/python3.10/dist-packages/zenml/orchestrators/step_launcher.py", line 499, in _run_step

AttributeError: imagen